In [4]:
import sys
if "../" not in sys.path: sys.path.insert(0,"../");

## Building utilities for marynlp

Creating multiple utilities for building models, downloading the files and a tone of other things

In [5]:
## Module downloading sequence

In [8]:
## Using the SED Morpheme template
from marynlp.utils.storage import download as dl
from marynlp.utils import storage

# setup the download bucket
bucket = storage.get_bucket("../resources/mary_africa_credentials_key.json", "marynlp-private")

morpheme_template_file = dl.file_from_google_temporary("models/sed_morpheme_template.txt", bucket); morpheme_template_file

'/tmp/tmp7onungq4/models/sed_morpheme_template.txt'